# House price regression

The main goal of this project is to estimate the sale price of real estate

## Problem
We got hired by a real estate investor in order to propose him a software solution to estimate quickly the value of housholds. The goal of this application is to scan the whole real estate market in order to indentify undervaluate household.

## Solution
To answer this problem, we propose a regression algorithm that will estimate the value of household given some specific properties. Then by comparing the estimate value against the market value, we'll be able to spot investment opportunities

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

### Data import, cleaning and analysis
The first step would be to import the data in order to prepare it for the algorithm.
To do so, we'll start by importing the data and list every feature

In [2]:
data = pd.read_csv('train.csv')
for name,dtype in zip(data.columns, data.dtypes):
    print(name,": ", dtype, "         Ex: ", data[name].iloc[0], "    Number of NaN: ", data[name].isnull().sum())

Id :  int64          Ex:  1     Number of NaN:  0
MSSubClass :  int64          Ex:  60     Number of NaN:  0
MSZoning :  object          Ex:  RL     Number of NaN:  0
LotFrontage :  float64          Ex:  65.0     Number of NaN:  259
LotArea :  int64          Ex:  8450     Number of NaN:  0
Street :  object          Ex:  Pave     Number of NaN:  0
Alley :  object          Ex:  nan     Number of NaN:  1369
LotShape :  object          Ex:  Reg     Number of NaN:  0
LandContour :  object          Ex:  Lvl     Number of NaN:  0
Utilities :  object          Ex:  AllPub     Number of NaN:  0
LotConfig :  object          Ex:  Inside     Number of NaN:  0
LandSlope :  object          Ex:  Gtl     Number of NaN:  0
Neighborhood :  object          Ex:  CollgCr     Number of NaN:  0
Condition1 :  object          Ex:  Norm     Number of NaN:  0
Condition2 :  object          Ex:  Norm     Number of NaN:  0
BldgType :  object          Ex:  1Fam     Number of NaN:  0
HouseStyle :  object          Ex: 

In [3]:
for column in data:
    print(data[column].value_counts()) 

1460    1
479     1
481     1
482     1
483     1
       ..
976     1
977     1
978     1
979     1
1       1
Name: Id, Length: 1460, dtype: int64
20     536
60     299
50     144
120     87
30      69
160     63
70      60
80      58
90      52
190     30
85      20
75      16
45      12
180     10
40       4
Name: MSSubClass, dtype: int64
RL         1151
RM          218
FV           65
RH           16
C (all)      10
Name: MSZoning, dtype: int64
60.0     143
70.0      70
80.0      69
50.0      57
75.0      53
        ... 
106.0      1
38.0       1
138.0      1
140.0      1
137.0      1
Name: LotFrontage, Length: 110, dtype: int64
7200     25
9600     24
6000     17
10800    14
9000     14
         ..
7094      1
6130      1
9337      1
5232      1
8190      1
Name: LotArea, Length: 1073, dtype: int64
Pave    1454
Grvl       6
Name: Street, dtype: int64
Grvl    50
Pave    41
Name: Alley, dtype: int64
Reg    925
IR1    484
IR2     41
IR3     10
Name: LotShape, dtype: int64
Lvl    1311


0      1436
168       3
216       2
144       2
180       2
245       1
238       1
290       1
196       1
182       1
407       1
304       1
162       1
153       1
320       1
140       1
130       1
96        1
23        1
508       1
Name: 3SsnPorch, dtype: int64
0      1344
192       6
224       5
120       5
189       4
       ... 
182       1
440       1
178       1
312       1
480       1
Name: ScreenPorch, Length: 76, dtype: int64
0      1453
738       1
648       1
576       1
555       1
519       1
512       1
480       1
Name: PoolArea, dtype: int64
Gd    3
Fa    2
Ex    2
Name: PoolQC, dtype: int64
MnPrv    157
GdPrv     59
GdWo      54
MnWw      11
Name: Fence, dtype: int64
Shed    49
Othr     2
Gar2     2
TenC     1
Name: MiscFeature, dtype: int64
0        1408
400        11
500         8
700         5
450         4
2000        4
600         4
1200        2
480         2
1150        1
800         1
15500       1
620         1
3500        1
560         1
2500        1


In [4]:
data_cleaned=data.drop(columns=["Id","Street","PoolQC","MiscFeature"] )
# on aurait aussi peut-être pu remplacer par la valeur qui apparait le plus souvent ou la moyenne
data_cleaned = data_cleaned[pd.notnull(data_cleaned['MasVnrType'])] 
#data_cleaned['MasVnrType']= data_cleaned['MasVnrType'].fillna('None') genre comme ça ou data['MasVnrType'].mean() si c'est float ou int
data_cleaned = data_cleaned[pd.notnull(data_cleaned['BsmtQual'])] 
data_cleaned = data_cleaned[pd.notnull(data_cleaned['BsmtExposure'])]
data_cleaned = data_cleaned[pd.notnull(data_cleaned['BsmtFinType2'])]
data_cleaned = data_cleaned[pd.notnull(data_cleaned['Electrical'])]

data_cleaned['Alley'] = data_cleaned['Alley'].fillna(0)
data_cleaned['Fence'] = data_cleaned['Fence'].fillna(0)



for name,dtype in zip(data_cleaned.columns, data_cleaned.dtypes):
    print(name,": ", dtype, "         Ex: ", data_cleaned[name].iloc[0], "    Number of NaN: ", data_cleaned[name].isnull().sum())

MSSubClass :  int64          Ex:  60     Number of NaN:  0
MSZoning :  object          Ex:  RL     Number of NaN:  0
LotFrontage :  float64          Ex:  65.0     Number of NaN:  253
LotArea :  int64          Ex:  8450     Number of NaN:  0
Alley :  object          Ex:  0     Number of NaN:  0
LotShape :  object          Ex:  Reg     Number of NaN:  0
LandContour :  object          Ex:  Lvl     Number of NaN:  0
Utilities :  object          Ex:  AllPub     Number of NaN:  0
LotConfig :  object          Ex:  Inside     Number of NaN:  0
LandSlope :  object          Ex:  Gtl     Number of NaN:  0
Neighborhood :  object          Ex:  CollgCr     Number of NaN:  0
Condition1 :  object          Ex:  Norm     Number of NaN:  0
Condition2 :  object          Ex:  Norm     Number of NaN:  0
BldgType :  object          Ex:  1Fam     Number of NaN:  0
HouseStyle :  object          Ex:  2Story     Number of NaN:  0
OverallQual :  int64          Ex:  7     Number of NaN:  0
OverallCond :  int64    

In [5]:

data['MSZoning'] = data['MSZoning'].replace("RH", 0).replace("FV", 1).replace("C", 2).replace("RM", 3).replace("RL", 4)
data['Alley'] = data['Alley'].replace("NA", 0).replace("Grvl", 1).replace("Pave", 2)
data['LotShape'] = data['LotShape'].replace("Reg", 3).replace("IR1", 2).replace("IR2", 1).replace("IR3", 0) #retrouver le petit quartier pour adapter les codes
data['LandContour'] = data['LandContour'].replace("Lvl", 0).replace("Bnk", 1).replace("HLS", 2).replace("Low", 3) #A voir si les valeur ont du sens
data['LotConfig'] = data['LotConfig'].replace("Inside", 0).replace("Corner", 1).replace("CulDSac", 2).replace("FR2", 3).replace("FR3", 4)
data['LandSlope'] = data['LandSlope'].replace("Sev", 0).replace("Mod", 1).replace("Gtl", 2)
data['Neighborhood'] = data['Neighborhood'].replace("NAmes", 0).replace("CollgCr").replace("OldTown") 
data['Condition1'] = data['Condition1'].replace("", 0) #Plus c'est haut mieux c'est
data['Condition2'] = data['Condition2'].replace("", 0) #la meme
data['BldgType'] = data['BldgType'].replace("2FmCon", 0).replace("Twnhs ",1).replace("Duplx", 2).replace("TwnhsE", 3).replace("1Fam", 4)
data['HouseStyle'] = data['HouseStyle'].replace("1Story", 0).replace("2Story1.5Fin", 1).replace("1.5Fin", 2).replace("SLvl", 3).replace("SFoyer", 4).replace("1.5Unf", 5).replace("2.5Unf", 6).replace("2.5Fin", 7)
data['RoofStyle'] = data['RoofStyle'].replace("Gable", 0).replace("Hip", 1).replace("Flat", 2).replace("Gambrel", 3).replace("Mansard", 4).replace("Shed", 5)
data['Exterior1st'] = data['Exterior1st'].replace("", 0) #A mettre dans l'odre 0 14 pas beaucoup
data['Exterior2nd'] = data['Exterior2nd'].replace("", 0) #A mettre dans l'odre 0 14 pas beaucoup
data['MasVnrType'] = data["MasVnrType"].replace("Nan", 0).replace("BrkFace", 1).replace("Stone", 2).replace("BrkCmn", 3) #Classer
#data['MasVnrArea'] = data['MasVnrArea'].replace("NaN", 0) #Mettre 0
data['ExterQual'] = data['ExterQual'].replace("TA", 0).replace("Gd", 1).replace("Ex", 2).replace("Fa", 3)
data['ExterCond'] = data['ExterQual'].replace("TA", 0).replace("Gd", 1).replace("Ex", 2).replace("Fa", 3).replace("Po", 4)
data['Foundation'] = data['Foundation'].replace("BrkCmn", 1).replace("BrkFace", 2).replace("CBlock", 3).replace("Stone", 4).replace("None", 0) #A adapter Ordre décroissant
data['BsmtQual'] = data['BsmtQual'].replace("Ex", 5).replace("Gd", 4).replace("TA", 3).replace("Fa", 2).replace("Po", 1).replace("Na", 0) #Droper NA et classer dans l'ordre 
data['BsmtCond'] = data['BsmtCond'].replace("Ex", 5).replace("Gd", 4).replace("TA", 3).replace("Fa", 2).replace("Po", 1).replace("Na", 0) # la même jusqu'à Total jusqu'au heat
data['BsmtExposure'] = data['BsmtExposure'].replace("Gd", 0).replace("Av", 0).replace("Mn", 0).replace("No", 0).replace("Na", 0)
data['BsmtFinType1'] = data['BsmtFinType1'].replace("GLQ", 6).replace("ALQ", 5).replace("BLQ", 4).replace("Rec", 3).replace("LwQ", 2).replace("Unf", 1).replace("NA", 0)
data['BsmtFinType2'] = data['BsmtFinType2'].replace("GLQ", 6).replace("ALQ", 5).replace("BLQ", 4).replace("Rec", 3).replace("LwQ", 2).replace("Unf", 1).replace("NA", 0)
#On recommence ici
data['Heating'] = data['Heating'].replace("Floor", 1).replace("GasA", 2).replace("GasW", 3).replace("Grav", 4).replace("OthW", 5).replace("Wall", 6) #Chaufage au sol meilleur note
data['HeatingQC'] = data['HeatingQC'].replace("Ex", 5).replace("Gd", 4).replace("TA", 3).replace("Fa", 2).replace("Po", 1) #classification croissant
data['CentralAir'] = data['CentralAir'].replace("Y", 1).replace("N", 0)
data['Electrical'] = data['Electrical'].replace("", 0) #virer la ligne NA et mettre les categorie dans l'ordre
data['KitchenQual'] = data['KitchenQual'].replace("Ex", 5).replace("Gd", 4).replace("TA", 3).replace("Fa", 2).replace("Po", 1) #dans l'0dre TA GD
data['Functional'] = data['Functional'].replace("", 0)
data['FireplaceQu'] = data['FireplaceQu'].replace("Ex", 5).replace("Gd", 4).replace("TA", 3).replace("Fa", 2).replace("Po", 1).replace("Na", 0)#mettre NA = 0 et classement
data['GarageType'] = data['GarageType'].replace("", 0) #mettre NA = 0 et classement
data['GarageYrBlt'] = data['GarageYrBlt'].replace("", 0) #mettre NA = 0 le reste c'est float
data['GarageFinish'] = data['GarageFinish'].replace("Fin", 3).replace("RFn", 2).replace("Unf", 1).replace("NA", 0)#mettre NA = 0 et classement
data['GarageQual'] = data['GarageQual'].replace("Ex", 5).replace("Gd", 4).replace("TA", 3).replace("Fa", 2).replace("Po", 1).replace("Na", 0) #mettre NA = 0 et classement
data['GarageCond'] = data['GarageCond'].replace("Ex", 5).replace("Gd", 4).replace("TA", 3).replace("Fa", 2).replace("Po", 1).replace("Na", 0) #mettre NA = 0 et classement
data['PavedDrive'] = data['PavedDrive'].replace("", 0) #Y et N Yes or No 1 ou 0
data['PoolQC'] = data['PoolQC'].replace("Ex", 5).replace("Gd", 4).replace("TA", 3).replace("Fa", 2).replace("Po", 1).replace("Na", 0) #Drop quality
data['Fence'] = data['Fence'].replace("", 0) #classement et mettre 0 à NaN
data['MiscFeature'] = data['MiscFeature'].replace("", 0) #drop
data['SaleType'] = data['SaleType'].replace("", 0) #classement
data['SaleCondition'] = data['SaleCondition'].replace("", 0) #classement

In [12]:
data.YearBuilt.mean() 

NameError: name 'mean' is not defined

Given this analysis, we can notice that some features doesn't contain any valuable information
Those features are:
- Id
- ...

In addition of that, we notice that there is many missing values. The job here would be to differanciate missing values (errors in the dataset) from the absence of the concernate feature in the house.

In the first case, we'll remove the row.

In the second case, we'll considere any object that can be absent as added value, and thus replacing NaNs by zero.

In [6]:
data.describe()

,Id,MSSubClass,LotFrontage,LotArea,Alley,LotShape,LandContour,LotConfig,LandSlope,OverallQual,...,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,MiscVal,MoSold,YrSold,SalePrice
count,1460.000000,1460.000000,1201.000000,1460.000000,91.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,...,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,7.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,730.500000,56.897260,70.049958,10516.828082,1.450549,2.591781,0.185616,0.416438,1.937671,6.099315,...,46.660274,21.954110,3.409589,15.060959,2.758904,3.714286,43.489041,6.321918,2007.815753,180921.195890
std,421.610009,42.300571,24.284752,9981.264932,0.500305,0.582296,0.606509,0.773448,0.276232,1.382997,...,66.256028,61.119149,29.317331,55.757415,40.177307,1.253566,496.123024,2.703626,1.328095,79442.502883
min,1.000000,20.000000,21.000000,1300.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,1.000000,2006.000000,34900.000000
25%,365.750000,20.000000,59.000000,7553.500000,1.000000,2.000000,0.000000,0.000000,2.000000,5.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000,0.000000,5.000000,2007.000000,129975.000000
50%,730.500000,50.000000,69.000000,9478.500000,1.000000,3.000000,0.000000,0.000000,2.000000,6.000000,...,25.000000,0.000000,0.000000,0.000000,0.000000,4.000000,0.000000,6.000000,2008.000000,163000.000000
75%,1095.250000,70.000000,80.000000,11601.500000,2.000000,3.000000,0.000000,1.000000,2.000000,7.000000,...,68.000000,0.000000,0.000000,0.000000,0.000000,4.500000,0.000000,8.000000,2009.000000,214000.000000
max,1460.000000,190.000000,313.000000,215245.000000,2.000000,3.000000,3.000000,4.000000,2.000000,10.000000,...,547.000000,552.000000,508.000000,480.000000,738.000000,5.000000,15500.000000,12.000000,2010.000000,755000.000000


In [7]:
data.describe(include=['O'])

,MSZoning,Street,Utilities,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,RoofMatl,Exterior1st,...,MasVnrType,Foundation,Electrical,Functional,GarageType,PavedDrive,Fence,MiscFeature,SaleType,SaleCondition
count,1460,1460,1460,1460,1460,1460,1460,1460,1460,1460,...,1452,1460,1459,1460,1379,1460,281,54,1460,1460
unique,5,2,2,24,9,8,5,8,8,15,...,4,6,5,7,6,3,4,4,9,6
top,4,Pave,AllPub,0,Norm,Norm,4,0,CompShg,VinylSd,...,None,PConc,SBrkr,Typ,Attchd,Y,MnPrv,Shed,WD,Normal
freq,1151,1454,1459,261,1260,1445,1220,726,1434,515,...,864,647,1334,1360,870,1340,157,49,1267,1198


#object_cols=[
#    'MSZoning' , 'Street', 'Alley', ' LotShape' , 
#    ' LandContour' , ' Utilities' , ' LotConfig' , ' LandSlope' , 
#    ' Neighborhood' , ' Condition1' , ' Condition2' , ' BldgType' , 
#    ' HouseStyle' , ' RoofStyle' , ' RoofMatl' , ' Exterior1st' , 
#    ' Exterior2nd' , ' MasVnrType' , ' ExterQual' , ' ExterCond' , 
#    ' Foundation' , ' BsmtQual' , ' BsmtCond' , ' BsmtExposure' , 
#    ' BsmtFinType1' , ' BsmtFinType2' , ' Heating' , ' HeatingQC' , 
#    ' CentralAir' , ' Electrical' , ' KitchenQual' , ' Functional' , 
#    ' FireplaceQu' , ' GarageType' , ' GarageFinish' , ' GarageQual' , 
#    ' GarageCond' , ' PavedDrive' , ' PoolQC' , ' Fence’ , ‘ MiscFeature' , 
#    ' SaleType' , ' SaleCondition'    
#]




















 








